In [1]:
import warnings

# Turn off all warnings (not recommended unless you're sure)
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.regularizers import l1, l2
from tensorflow.keras.metrics import Precision, Recall

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt


In [3]:
!pwd

/Users/joshstone/code/lucasglanville/and_theyre_off_backend


In [4]:
import pandas as pd
import os
#from params import *


def get_data(LOCAL_FILEPATH):
    data = pd.read_csv(LOCAL_FILEPATH)
    print(f"data acquired with shape {data.shape}")
    return data



#This is the first 2 days worth of data where general_runs stats are all skewed
def remove_221_rows(data):
    data = data[221:]
    print(f"first 221 rows removed. New shape = {data.shape}")
    return data

## Dropping rows if we have no betting data, and imputing if we have some betting data
def dropping_no_betting_data(data):
    data = data.dropna(subset=['f_bsp', 'f_pm_15m', 'f_pm_10m', 'f_pm_05m' , 'f_pm_03m', 'f_pm_02m', 'f_pm_01m'], how='all')
    def impute_row(row):
        row = row.fillna(method='ffill').fillna(method='bfill')
        return row
    columns_to_impute = ['f_bsp', 'f_pm_15m', 'f_pm_10m', 'f_pm_05m', 'f_pm_03m', 'f_pm_02m', 'f_pm_01m']
    data[columns_to_impute] = data[columns_to_impute].apply(impute_row, axis=1)

    #deleting row that has 0s for all odds for some reason


    print(f"Cleaned up missing odds. New shape = {data.shape}")
    return data

def josh_features(data):

    #Creating country feature
    irish_tracks = [
    "SLIGO", "LIMERICK", "NAVAN", "WEXFORD", "CURRAGH",
    "GALWAY", "KILBEGGAN", "GOWRAN PARK", "BELLEWSTOWN",
    "LISTOWEL", "THURLES", "BALLINROBE", "TRAMORE",
    "LEOPARDSTOWN", "DOWN ROYAL", "ROSCOMMON", "CORK",
    "DUNDALK", "KILLARNEY", "LAYTOWN", "TIPPERARY",
    "FAIRYHOUSE", "NAAS", "DOWNPATRICK", "CLONMEL",
    "PUNCHESTOWN"
]

    data['country'] = data['f_track'].apply(lambda x: 'IRE' if x in irish_tracks else 'GB')

    #Completing f_class for Irish races

    # Calculate mean ratings for each 'f_id' group
    mean_ratings_by_id = data.groupby('f_id')['f_rating_or'].mean()

    # Define the mapping of mean ratings to f_class values
    rating_to_f_class_mapping = {
        (96, float('inf')): 1,
        (86, 96): 2,
        (76, 86): 3,
        (66, 76): 4,
        (56, 66): 5,
        (46, 56): 6,
        (-float('inf'), 46): 7
    }

    # Function to map mean ratings to f_class values
    def map_rating_to_f_class(mean_rating):
        for rating_range, f_class_value in rating_to_f_class_mapping.items():
            if rating_range[0] <= mean_rating <= rating_range[1]:
                return f_class_value

    # Apply the mapping to fill NULL values in 'f_class' column based on mean ratings
    data['f_class'] = data.apply(lambda row: map_rating_to_f_class(mean_ratings_by_id.get(row['f_id'])), axis=1)

    # Now the 'f_class' column should be filled based on the specified mapping using mean ratings

    # Merge the mean ratings back into the original DataFrame based on 'f_id'
    data = data.merge(mean_ratings_by_id, how='left', left_on='f_id', right_index=True)

    # Rename the merged mean rating column for clarity
    data.rename(columns={'f_rating_or_y': 'mean_f_rating_or_race', 'f_rating_or_x' : 'f_rating_or' }, inplace=True)

    # Create official rating vs average rating in the race feature
    data['or_rating_vs_avg_race'] = data['f_rating_or'] - data['mean_f_rating_or_race']

    # Create odds percentage and movement features
    data['15m_odds_prob'] = 1 / data['f_pm_15m']
    data['5m_odds_prob'] = 1 / data['f_pm_05m']
    data['15to5m_odds_move_perc'] = (data['5m_odds_prob'] / data['15m_odds_prob'] - 1)
    data['15to5m_odds_move_raw'] = (data['5m_odds_prob'] - data['15m_odds_prob'])
    print(f"Added Josh features. New shape = {data.shape}")
    return data

def class_or_rating_average(data):
    average_or_rating_class = data.groupby("f_class")['f_rating_or'].mean().reset_index()
    average_or_rating_class.rename(columns={'f_rating_or': 'average_or_rating_class'}, inplace=True)
    data = data.merge(average_or_rating_class, on='f_class')
    data['above_below_official_rating_class'] = data['f_rating_or']- data['average_or_rating_class']
    print(f"Added Oli features 2/4. New shape = {data.shape}")
    return data


def oli_features(data):
    data = data.sort_values(by='f_ko')
    data['PreviousPosition'] = data.groupby('f_horse')['f_place'].shift(fill_value=0)

    data = data.sort_values(by=['f_id', 'pred_isp'])
    data['PredictedRank'] = data.groupby('f_id').cumcount() + 1
    print(f"Added Oli features 4/4. New shape = {data.shape}")
    return data

# Define a lambda function to apply the conditions and fill NaN values
def fill_f_pm_01m(data):
    def fill_nan(row):
        if row['f_place'] == 0:
            return -1
        elif row['f_place'] == 1:
            return (row['f_pm_01m'] - 1) * 0.95
        else:
            return row['f_pm_01m_p_back']

# Apply the lambda function to fill NaN values in f_pm_01m_p_back
    data['linear_target'] = data.apply(fill_nan, axis=1)
    return data

data = get_data("raw_data/raw_data_v2.2.csv")
data = remove_221_rows(data)
data = dropping_no_betting_data(data)
    #Dropping line that has 0 for all odds
data = data[data.id != 16157910000382]
data = josh_features(data)
data = class_or_rating_average(data)
data = oli_features(data)
data.to_csv("raw_data/data_clean.csv", index=False)

data acquired with shape (118575, 116)
first 221 rows removed. New shape = (118354, 116)
Cleaned up missing odds. New shape = (118093, 116)
Added Josh features. New shape = (118092, 123)
Added Oli features 2/4. New shape = (118092, 125)
Added Oli features 4/4. New shape = (118092, 127)


In [ ]:
import numpy as np
import pandas as pd

from sklearn import set_config
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer

In [ ]:
def preprocess_features_v2(data: pd.DataFrame) -> np.ndarray:
    #--------------------------------------------------------------------------#
    print('number of columns: ', len(data.columns))
    ### DROP IRRELEVANT COLUMNS ###
    redundant_columns = [
                         #"id",
                         #"f_id",
                         "f_racetype",
                         #"f_horse",
                         "f_jockey",
                         "f_trainer",
                         "f_rating_hc",
                         "f_lto_pos",
                         "f_bsp",
                         #"f_pm_15m",
                         "f_pm_10m",
                         "f_pm_05m",
                         "f_pm_03m",
                         "f_pm_02m",
                         #"f_pm_01m",
                         "f_bsp_p_back",
                         "f_bsp_p_lay",
                         #"f_pm_01m_p_back",
                         "f_pm_01m_p_lay",
                         #"f_pm_15m_p_back",
                         "f_pm_15m_p_lay",
                         "general_runs_win_at",
                         "general_runs_win_l200r",
                         "general_runs_win_l50r",
                         "general_runs_win_l16r",
                         "general_runs_at",
                         "general_runs_l200r",
                         "general_runs_l50r",
                         "general_runs_l16r",
                         "sum_bsp_trainer_at",
                         "sum_bsp_jockey_at",
                         "sum_bsp_horse_at",
                         "sum_bsp_trainer_l16r",
                         "sum_bsp_jockey_l16r",
                         "sum_bsp_trainer_l50r",
                         "sum_bsp_jockey_l50r",
                         "sum_bsp_trainer_l200r",
                         "sum_bsp_jockey_l200r",
                         "sum_bsp_horse_l10r",
                         "sum_bsp_horse_l5r",
                         "sum_bsp_horse_l2r",
                         "15to5m_odds_move_perc",
                         "15to5m_odds_move_raw",
                         "15m_odds_prob",
                         "5m_odds_prob"]
    data.drop(columns = redundant_columns, inplace = True)
    print("✅ DROPPED IRRELEVANT COLUMNS")

    #--------------------------------------------------------------------------#

    ### DROP ROWS WITH NULL VALUES IN THESE COLUMNS ###
    data.dropna(axis = 0, inplace = True,
                subset = ["f_pace",
                          "f_stall",
                          "stall_position",
                          "f_going",
                          "f_rating_or",
                          "or_rating_vs_avg_race",
                          "country",
                          "mean_f_rating_or_race",
                          "f_class",
                          "average_or_rating_class",
                          "above_below_official_rating_class",
                          "PreviousPosition",
                          "PredictedRank",
                          ])
    print("✅ DROPPED ROWS WITH NULL VALUES")

    #--------------------------------------------------------------------------#

    ### STRIP SURROUNDING WHITESPACE ###
    data['f_track'] = data['f_track'].str.strip()
    print("✅ WHITESPACE STRIPPED FROM 'f_track'")

    #--------------------------------------------------------------------------#

    ### CONVERT ODDS TO PROBABILITY ###
    def odds_to_prob(x):
        return 1/x
    data['pred_isp'] = data['pred_isp'].apply(odds_to_prob)
    print("✅ ODDS CONVERTED TO PROBABILITY (1/ODDS)")

    #--------------------------------------------------------------------------#

    ### CODE WINNERS AS '1', REST AS '0' ###
    def winner(x):
        if x == 1:
            return 1
        else:
            return 0
    data['f_place'] = data['f_place'].apply(winner)
    print("✅ WINNERS CODED AS '1', REST '0'")

    #--------------------------------------------------------------------------#

    ### 'f_ko' CONVERTED TO DATETIME ###
    data['f_ko'] = data['f_ko'].astype('datetime64[ns]')
    print("✅ 'f_ko' CONVERTED TO DATETIME")

    #--------------------------------------------------------------------------#

    ### ENCODE THE GOING (GROUND CONDITION) ###
    def f_going_coder(x):
        if x == 'FRM':
            return 1
        elif x == 'GTF':
            return 2
        elif x == 'GD' or x == 'GTY' or x == 'STD':
            return 3
        elif x == 'YLD' or x == 'YTS' or x == 'STSL' or x == 'GTS':
            return 4
        elif x == 'SFT':
            return 5
        elif x == 'HVY' or x == 'HTS':
            return 6
    data['f_going'] = data['f_going'].apply(f_going_coder)
    print("✅ TRACK CONDITIONS ORDINALLY ENCODED")

    #--------------------------------------------------------------------------#

    ### MINMAX SCALE NUMERIC FEATURES ###
    set_config(transform_output = "pandas")
    numeric_features = ["f_going",
                        "average_or_rating_class",
                        "above_below_official_rating_class",
                        "PreviousPosition",
                        "PredictedRank",
                        "f_distance",
                        "f_class",
                        "f_age",
                        "f_pace",
                        "f_weight",
                        "f_runners",
                        "f_rating_or",
                        "mean_f_rating_or_race",
                        "or_rating_vs_avg_race",
                        "f_rating_rbd",
                        "f_stall",
                        "stall_position",
                        "trainer_runs_win_at",
                        "trainer_runs_win_l200r",
                        "trainer_runs_win_l50r",
                        "trainer_runs_win_l16r",
                        "trainer_runs_at",
                        "trainer_runs_l200r",
                        "trainer_runs_l50r",
                        "trainer_runs_l16r",
                        "jockey_runs_win_at",
                        "jockey_runs_win_l200r",
                        "jockey_runs_win_l50r",
                        "jockey_runs_win_l16r",
                        "jockey_runs_at",
                        "jockey_runs_l200r",
                        "jockey_runs_l50r",
                        "jockey_runs_l16r",
                        "horse_runs_win_at",
                        "horse_runs_win_l10r",
                        "horse_runs_win_l5r",
                        "horse_runs_win_l2r",
                        "horse_runs_at",
                        "horse_runs_l10r",
                        "horse_runs_l5r",
                        "horse_runs_l2r",
                        "iv_horse_at",
                        "iv_trainer_l200r",
                        "iv_trainer_l50r",
                        "iv_trainer_l16r",
                        "iv_trainer_at",
                        "iv_jockey_l200r",
                        "iv_jockey_l50r",
                        "iv_jockey_l16r",
                        "iv_jockey_at",
                        "ae_horse_l10r",
                        "ae_horse_l5r",
                        "ae_horse_l2r",
                        "ae_horse_at",
                        "ae_trainer_l200r",
                        "ae_trainer_l50r",
                        "ae_trainer_l16r",
                        "ae_trainer_at",
                        "ae_jockey_l200r",
                        "ae_jockey_l50r",
                        "ae_jockey_l16r",
                        "ae_jockey_at",
                        "rolling_avg_trainer_finish_at",
                        "rolling_avg_trainer_finish_l200r",
                        "rolling_avg_trainer_finish_l50r",
                        "rolling_avg_trainer_finish_l16r",
                        "rolling_avg_horse_finish_at",
                        "rolling_avg_horse_finish_l10r",
                        "rolling_avg_horse_finish_l5r",
                        "rolling_avg_horse_finish_l2r",
                        "rolling_avg_jockey_finish_at",
                        "rolling_avg_jockey_finish_l200r",
                        "rolling_avg_jockey_finish_l50r",
                        "rolling_avg_jockey_finish_l16r",
                        ]
    numeric_transformer = Pipeline(
        steps=[("scaler", MinMaxScaler())])
    print("✅ NUMERIC FEATURES MINMAX-SCALED")

    #--------------------------------------------------------------------------#

    ### IMPUTE HEADGEAR NULLS WITH 'no_headgear' ###
    headgear_feature = ["f_headgear"]
    headgear_imputer = Pipeline(
        steps=[("imputer", SimpleImputer(strategy = 'constant',
                                         fill_value = 'no_headgear'))])
    print("✅ IMPUTED 'no_headgear' for NULLS IN 'f_headgear'")

    #--------------------------------------------------------------------------#

    ### MEAN IMPUTE CERTAIN FEATURES ###
    mean_impute_features = ["f_dob", "f_prb_avg"]
    mean_imputer = Pipeline(
        steps=[("imputer", SimpleImputer(strategy = 'mean'))])
    print("✅ IMPUTED MEAN FOR NULLS IN 'f_dob' & 'f_prb_avg'")

    #--------------------------------------------------------------------------#

    ### IMPUTE NULLS WITH 0's ###
    zero_impute_features = ["f_rating_rbd",
                            "trainer_runs_win_at",
                            "trainer_runs_win_l200r",
                            "trainer_runs_win_l50r",
                            "trainer_runs_win_l16r",
                            "trainer_runs_at",
                            "trainer_runs_l200r",
                            "trainer_runs_l50r",
                            "trainer_runs_l16r",
                            "jockey_runs_win_at",
                            "jockey_runs_win_l200r",
                            "jockey_runs_win_l50r",
                            "jockey_runs_win_l16r",
                            "jockey_runs_at",
                            "jockey_runs_l200r",
                            "jockey_runs_l50r",
                            "jockey_runs_l16r",
                            "horse_runs_win_at",
                            "horse_runs_win_l10r",
                            "horse_runs_win_l5r",
                            "horse_runs_win_l2r",
                            "horse_runs_at",
                            "horse_runs_l10r",
                            "horse_runs_l5r",
                            "horse_runs_l2r",
                            "iv_horse_at",
                            "iv_trainer_l200r",
                            "iv_trainer_l50r",
                            "iv_trainer_l16r",
                            "iv_trainer_at",
                            "iv_jockey_l200r",
                            "iv_jockey_l50r",
                            "iv_jockey_l16r",
                            "iv_jockey_at",
                            "ae_horse_l10r",
                            "ae_horse_l5r",
                            "ae_horse_l2r",
                            "ae_horse_at",
                            "ae_trainer_l200r",
                            "ae_trainer_l50r",
                            "ae_trainer_l16r",
                            "ae_trainer_at",
                            "ae_jockey_l200r",
                            "ae_jockey_l50r",
                            "ae_jockey_l16r",
                            "ae_jockey_at",
                            "rolling_avg_trainer_finish_at",
                            "rolling_avg_trainer_finish_l200r",
                            "rolling_avg_trainer_finish_l50r",
                            "rolling_avg_trainer_finish_l16r",
                            "rolling_avg_horse_finish_at",
                            "rolling_avg_horse_finish_l10r",
                            "rolling_avg_horse_finish_l5r",
                            "rolling_avg_horse_finish_l2r",
                            "rolling_avg_jockey_finish_at",
                            "rolling_avg_jockey_finish_l200r",
                            "rolling_avg_jockey_finish_l50r",
                            "rolling_avg_jockey_finish_l16r"]
    zero_imputer = Pipeline(
        steps=[("imputer", SimpleImputer(strategy = 'constant',
                                         fill_value = 0))])
    print("✅ IMPUTED '0' FOR NULLS IN 68 x FEATURES")

    #--------------------------------------------------------------------------#

    ### O.H.E. CATEGORICAL FEATURES ###
    categorical_features = ["f_track", "f_headgear", "country"]
    categorical_transformer = Pipeline(
        steps=[("encoder", OneHotEncoder(handle_unknown="ignore",
                                     sparse_output=False))])
    print("✅ CAT. FEATURES OH-ENCODED (Track, Headgear, Country)")

    #--------------------------------------------------------------------------#

    ### COLUMN TRANSFORMER ###
    ct1 = ColumnTransformer(
        transformers=[("zero_imputer", zero_imputer, zero_impute_features),
                      ("headgear_imputer", headgear_imputer, headgear_feature),
                      ("mean_imputer", mean_imputer, mean_impute_features),
                      ],
        verbose_feature_names_out = False,
        remainder = 'passthrough')

    ct1_processed = ct1.fit_transform(data)

    print('number of columns: ', len(ct1_processed.columns))

    ct2 = ColumnTransformer(
        transformers=[("cat", categorical_transformer, categorical_features)],
        verbose_feature_names_out = False,
        remainder = 'passthrough')

    ct2_processed = ct2.fit_transform(ct1_processed)

    ct3 = ColumnTransformer(
        transformers=[("scale", numeric_transformer, numeric_features),],
        verbose_feature_names_out = False,
        remainder = 'passthrough')

    print("✅ COLUMN TRANSFORMER ASSEMBLED")

    #--------------------------------------------------------------------------#

    ### FIT_TRANSFORM FEATURES ###
    print("⏳ FIT_TRANSFORMING THE PREPROCESSING PIPE...")
    data_processed = ct3.fit_transform(ct2_processed)
    print('number of columns: ', len(data_processed.columns))
    print("✅ DATA PROCESSED WITH SHAPE:", data_processed.shape)

    return data_processed

In [ ]:
data = preprocess_features_v2(data)

In [ ]:
data = pd.read_csv("raw_data/data_cleaned_and_preprocessed.csv")

In [ ]:
data = data.sort_values(by='f_ko')

In [ ]:
data=data.query("f_pm_01m < 15 ")

In [ ]:
data = data.reset_index(drop=True)

In [ ]:
data.shape

In [ ]:
# X_train = train.iloc[:24882]
# X_val = X.iloc[24882:49544]
# X_test = X.iloc[49544:]
# y_train = y.iloc[:24882]
# y_val = y.iloc[24882:49544]
# y_test = y.iloc[49544:]
# backtest_train = backtest.iloc[:24882]
# backtest_val = backtest.iloc[24882:49544]
# backtest_test = backtest.iloc[49544:]

In [ ]:
# Split the data chronologically to avoid look-ahead bias:
test_start_date = '2022-08-20 18:45:00'  # max_date - 1 yr
val_start_date = '2022-02-20 18:45:00'  # max_date - 1.5 yrs
# Split the data into test and train
train = data[data['f_ko'] <= val_start_date]
val = data[(data['f_ko'] > val_start_date) & (data['f_ko'] <= test_start_date)]
test = data[data['f_ko'] > test_start_date]
# Split the data into X and y
X_train = train.drop(columns=["f_place", "f_pm_15m", "f_pm_05m", "f_pm_01m", "f_pm_15m_p_back", "f_id", "f_ko"])
X_val = val.drop(columns = ["f_place", "f_pm_15m", "f_pm_05m", "f_pm_01m", "f_pm_15m_p_back", "f_id", "f_ko"])
X_test = test.drop(columns = ["f_place", "f_pm_15m", "f_pm_05m", "f_pm_01m", "f_pm_15m_p_back", "f_id", "f_ko"])
y_train = train["f_place"]
y_val = val["f_place"]
y_test = test["f_place"]
len(X_train), len(X_val), len(X_test), len(y_train), len(y_val), len(y_test)

In [ ]:
X_val

In [ ]:
backtesting= data[data['f_ko'] > test_start_date]

In [ ]:
# X_train=X.iloc[:73753]
# X_val=X.iloc[73753:91432]
# X_test=X.iloc[91432:]
# y_train=y.iloc[:73753]
# y_val=y.iloc[73753:91432]
# y_test=y.iloc[91432:]

In [ ]:
# pca = PCA()

# # Fit PCA on your data
# pca.fit(X_train)

# # Get explained variance ratio
# explained_variance_ratio = pca.explained_variance_ratio_

# # Calculate cumulative explained variance
# cumulative_variance = np.cumsum(explained_variance_ratio)

# # Plot explained variance ratio
# plt.figure(figsize=(10, 6))
# plt.plot(range(1, len(cumulative_variance) + 1), cumulative_variance, marker='o')
# plt.title('Cumulative Explained Variance')
# plt.xlabel('Number of Components')
# plt.ylabel('Cumulative Explained Variance')
# plt.grid(True)
# plt.show()

In [ ]:
pca = PCA(n_components=70)

In [ ]:
X_train_pca = pca.fit_transform(X_train)

In [ ]:
X_val_pca = pca.transform(X_val)

In [ ]:
X_test_pca = pca.transform(X_test)

In [ ]:
# def custom_loss(implied_probabilities, bet_amount, threshold=0.06):
#     def loss(y_true, y_pred):
#         # Calculate the potential profit/loss for each prediction
#         bet_placed = y_pred > (implied_probabilities + threshold)
#         profit_loss = tf.where(y_true == 1, tf.where(bet_placed, (implied_probabilities-1) * bet_amount * y_pred, 0)
#                                , tf.where(bet_placed,bet_amount * y_pred, 0))

#         # Calculate the binary cross-entropy loss
#         binary_crossentropy = tf.keras.losses.BinaryCrossentropy(from_logits=False)(y_true, y_pred)

#         # Combine all loss components
#         custom_loss_value = tf.reduce_mean(binary_crossentropy + profit_loss)

#         return custom_loss_value
#     return loss

In [ ]:

def build_model(input_dim):
    model = Sequential()

    # Input layer
    model.add(Dense(64, input_dim=input_dim, activation='relu'))
    model.add(Dropout(0.15))

    # Hidden layers
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.01))

    model.add(Dense(64, activation='relu'))
    
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.1))
    
    # Output layer
    model.add(Dense(1, activation='sigmoid'))

    return model

model = build_model(X_train_pca.shape[1])


In [ ]:

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.00005), loss="binary_crossentropy", metrics=[Precision()])

In [ ]:

# Define callbacks (optional)
early_stopping = EarlyStopping(patience=20, restore_best_weights=True)


In [ ]:

# Train the model
history = model.fit(X_train_pca, y_train, batch_size=16, epochs=1000, validation_data=(X_val_pca, y_val), callbacks=[early_stopping])


In [ ]:
y_pred = model.predict(X_test_pca)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sns.histplot(y_pred, bins=50, kde=True);  # Adjust the number of bins as needed


In [ ]:
backtesting["y_pred"]=y_pred

In [ ]:
backtesting["probabilities"]=1/backtesting["f_pm_15m"]

In [ ]:
backtesting["diff"] = backtesting["y_pred"] - backtesting["probabilities"]

In [ ]:
backtesting["diff"].mean()

In [ ]:
sns.histplot(backtesting["diff"], bins=20, kde=True); 

In [ ]:
backtesting["bet_0.1"] = backtesting.apply(lambda row: 1 if row['y_pred'] - row['probabilities'] > 0.1 else 0, axis=1)

In [ ]:
backtesting["bet_0.05"] = backtesting.apply(lambda row: 1 if row['y_pred'] - row['probabilities'] > 0.05 else 0, axis=1)

In [ ]:
backtesting["bet_0.15"] = backtesting.apply(lambda row: 1 if row['y_pred'] - row['probabilities'] > 0.15 else 0, axis=1)

In [ ]:
backtesting["bet_0.05"].value_counts()

In [ ]:
backtesting["bet_0.1"].value_counts()

In [ ]:
backtesting['bet_0.15'].value_counts()

In [ ]:
backtesting['bets_placed_0.15'] = backtesting['bet_0.15'].cumsum()
backtesting['bets_placed_0.1'] = backtesting["bet_0.1"].cumsum()
backtesting['bets_placed_0.05'] = backtesting['bet_0.05'].cumsum()

In [ ]:
backtesting['profit_0.05']=backtesting['bet_0.05'] * backtesting["f_pm_15m_p_back"]
backtesting['profit_0.1']=backtesting['bet_0.1'] * backtesting["f_pm_15m_p_back"]
backtesting['profit_0.15']=backtesting['bet_0.15'] * backtesting["f_pm_15m_p_back"]

In [ ]:
plt.figure(figsize=(10, 6))  # Set the figure size (width, height)
plt.plot(backtesting['bets_placed_0.05'], backtesting['profit_0.05'].cumsum(), linestyle='-', color='b', label='5% threshold')
plt.plot(backtesting['bets_placed_0.1'], backtesting['profit_0.1'].cumsum(), linestyle='-', color='r', label='10% threshold')
plt.plot(backtesting['bets_placed_0.15'], backtesting['profit_0.15'].cumsum(), linestyle='-', color='g', label='15% threshold')
plt.title('Profit Over Time')
plt.xlabel('Time')
plt.ylabel('Profit')
plt.grid(True)

# tick_interval = pd.DateOffset(days=30)  
# start_date = backtesting['f_ko'].min()
# end_date = backtesting['f_ko'].max()
# x_ticks = pd.date_range(start=start_date, end=end_date, freq=tick_interval)
# plt.xticks(x_ticks, x_ticks.strftime('%b %d')) 

# plt.xticks(rotation=45)  


plt.legend()
plt.show()

In [ ]:
backtesting['profit_0.05'].cumsum()

In [ ]:
backtesting['profit_0.1'].cumsum()

In [ ]:
backtesting['profit_0.15'].cumsum()

In [ ]:
# model.save_weights("model_001")

In [ ]:
# pd.set_option('display.max_columns', None)

In [ ]:
# X_test['f_ko'] = X_test['f_ko'].astype(int) // 10**9

In [ ]:
# X_train['f_ko'] = pd.to_datetime(X_train['f_ko'])

In [ ]:
# X_test['f_ko']

In [ ]:
# scaler = StandardScaler()

In [ ]:
# X_test['f_ko'] = scaler.fit_transform(X_test[['f_ko']])

In [ ]:
# X_test['f_ko']

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history["val_loss"])

plt.show()
    